<a href="https://colab.research.google.com/github/cafechungkhoan/chu_gia/blob/master/Auto_ML_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaret

In [15]:
from google.colab import files
uploaded = files.upload()

Saving anomaly detection vnindex.xls to anomaly detection vnindex (1).xls


In [22]:
import pandas as pd
import io
data = pd.read_excel(io.BytesIO(uploaded['anomaly detection vnindex.xls']))
print(data)

*** No CODEPAGE record, no encoding_override: will use 'ascii'
            Date      Time        Close
0     2021/07/08  09:00:00  1374.680054
1     2021/07/07  09:00:00  1388.550000
2     2021/07/06  09:00:00  1354.790000
3     2021/07/05  09:00:00  1411.130000
4     2021/07/02  09:00:00  1420.270000
...          ...       ...          ...
2997  2009/07/07  09:00:00   447.630000
2998  2009/07/06  09:00:00   454.270000
2999  2009/07/03  09:00:00   435.440000
3000  2009/07/02  09:00:00   433.790000
3001  2009/07/01  09:00:00   430.000000

[3002 rows x 3 columns]


In [23]:
data['Date'] = pd.to_datetime(data['Date'])
data = data.drop(['Time'],axis = 1)
data = data.dropna()
data

,Date,Close
0,2021-07-08,1374.680054
1,2021-07-07,1388.550000
2,2021-07-06,1354.790000
3,2021-07-05,1411.130000
4,2021-07-02,1420.270000
...,...,...
2997,2009-07-07,447.630000
2998,2009-07-06,454.270000
2999,2009-07-03,435.440000
3000,2009-07-02,433.790000


In [24]:
# set timestamp to index
data.set_index('Date', drop=True, inplace=True)
# creature features from date
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]

In [25]:
data

,Close,day,day_name,day_of_year,week_of_year,hour,is_weekday
Date,,,,,,,
2021-07-08,1374.680054,8,Thursday,189,27,0,4
2021-07-07,1388.550000,7,Wednesday,188,27,0,3
2021-07-06,1354.790000,6,Tuesday,187,27,0,2
2021-07-05,1411.130000,5,Monday,186,27,0,1
2021-07-02,1420.270000,2,Friday,183,26,0,5
...,...,...,...,...,...,...,...
2009-07-07,447.630000,7,Tuesday,188,28,0,2
2009-07-06,454.270000,6,Monday,187,28,0,1
2009-07-03,435.440000,3,Friday,184,27,0,5


In [26]:
# init setup
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,session_id,123
1,Original Data,"(3002, 7)"
2,Missing Values,False
3,Numeric Features,4
4,Categorical Features,3
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(3002, 15)"
9,CPU Jobs,-1


In [27]:
# check list of available models
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [28]:
# train model
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

,Close,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
Date,,,,,,,,,
2021-07-08,1374.680054,8,Thursday,189,27,0,4,0,-0.002499
2021-07-07,1388.550000,7,Wednesday,188,27,0,3,1,0.026437
2021-07-06,1354.790000,6,Tuesday,187,27,0,2,1,0.014143
2021-07-05,1411.130000,5,Monday,186,27,0,1,1,0.037205
2021-07-02,1420.270000,2,Friday,183,26,0,5,1,0.030987


In [29]:
# check anomalies
iforest_results[iforest_results['Anomaly'] == 1].head()

,Close,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
Date,,,,,,,,,
2021-07-07,1388.55,7,Wednesday,188,27,0,3,1,0.026437
2021-07-06,1354.79,6,Tuesday,187,27,0,2,1,0.014143
2021-07-05,1411.13,5,Monday,186,27,0,1,1,0.037205
2021-07-02,1420.27,2,Friday,183,26,0,5,1,0.030987
2021-07-01,1417.08,1,Thursday,182,26,0,4,1,0.024050


In [32]:
import plotly.graph_objects as go
# plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y="Close", title='VnIndex - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['Close'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()